# Dtron2 Inference

## Imports

In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
# opencv is pre-installed on colab
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import gc
#del variables
gc.collect()
# Gong added this:
torch.cuda.empty_cache()
#torch.cuda.memory_summary(device=None, abbreviated=False)

# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
import json
import math
import os
import shutil 
import json
import re

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, VisImage
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

from detectron2.utils.visualizer import ColorMode
import glob

import random
from detectron2.utils.visualizer import Visualizer
import numpy as np 
import os


/home/spenc/miniconda3/envs/dtron2_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.10.1+cu111 True


## Functions

In [2]:
def get_soi(str1, start_char, end_char):
    str1 = str(str1)
    offst = len(start_char)
    ind1 = str1.find(start_char)
    ind2 = str1.find(end_char)
    s_str = str1[ind1+offst:ind2]
    return s_str

In [3]:
def createDataDict (fn, outputs):
	img_shape = list(outputs["instances"].image_size)
	img_h = int(img_shape[0])
	img_w = int(img_shape[1])

	class_list = get_soi(outputs["instances"].pred_classes, "[", "]").split(",")
	class_list_new = []
	for each in class_list:
		class_list_new.append(int(each.strip()))

	bbox_list = get_soi(outputs["instances"].pred_boxes, "[[", "]]").split("]")
	bbox_list_new = []
	for each in bbox_list:
		bbox = re.sub("['[,\n]", "", each).split(" ")
		bbox_new = []
		for item in bbox:
			if item != "":
				bbox_new.append(float(item))
		bbox_list_new.append(bbox_new)

	ann_list = []
	for i in range(0, len(class_list)):
		# og was "bbox_mode": "<BoxMode.XYWH_ABS: 1>"
		ann_list.append({"iscrowd": 0, "bbox": bbox_list_new[i], "category_id": class_list_new[i], "bbox_mode": 0})
	
	data_dict = {
		"file_name": fn,
		"height": img_h,
		"width": img_w, 
		"annotations": ann_list
	}
 
	return data_dict

## Inference with D2 Saved Weights

In [4]:
### File paths

img_out_dir = "./img_out/"
img_in_dir = "./test_inf_imgs/"
results_dir = "./results"

if not os.path.exists(img_out_dir):
    os.makedirs(img_out_dir)
    
if not os.path.exists(img_in_dir):
    os.makedirs(img_in_dir)
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
### Starting inference

print ("model inference started...")

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = os.path.join("./model_pth/", "model_final.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 32 + 1 #your number of classes + 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
predictor = DefaultPredictor(cfg)

img_path_list = os.listdir(img_in_dir)

master_dict = []

for img_path in img_path_list:
	img = cv2.imread(img_in_dir + img_path)
	outputs = predictor(img)
	print(outputs)
	data_dict = createDataDict(img_in_dir + img_path, outputs)
	vis = Visualizer(img[:, :, ::-1], scale=1)
	out = vis.draw_dataset_dict(data_dict)
	cv2.imwrite("./img_out/"+img_path, out.get_image()[:, :, ::-1])
	master_dict.append(data_dict)
	with open("./results/data_dict.json", "w+") as f:
		f.write(json.dumps(master_dict))

model inference started...
[09/03 22:43:04 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./model_pth/model_final.pth ...


/home/spenc/miniconda3/envs/dtron2_venv/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


{'instances': Instances(num_instances=16, image_height=1080, image_width=1920, fields=[pred_boxes: Boxes(tensor([[ 682.7680,   77.0000,  758.9775,  108.5063],
        [ 611.7362,   77.0773,  666.5400,  107.0134],
        [ 525.1709,   79.6921,  591.4139,  105.4467],
        [1405.6550,   62.5482, 1506.3121,  117.7863],
        [1120.8167,  986.6182, 1296.1089, 1023.8476],
        [ 608.5143,   64.3737,  669.5109,  120.8887],
        [1307.6422,  989.6331, 1472.8630, 1021.3987],
        [1352.9832,    8.4826, 1519.4431,   40.9275],
        [ 514.2072,   65.7212,  594.8746,  122.8667],
        [ 976.0560,  841.2876, 1190.6113,  881.1710],
        [ 925.5764,  985.5916, 1106.5454, 1026.2626],
        [1339.0411,   81.3475, 1382.8290,  103.7738],
        [ 404.9393,    6.7794,  434.5810,   41.8670],
        [ 401.1217,   54.9912,  455.3057,  131.3951],
        [1487.7677,  903.3296, 1510.8457,  926.3126],
        [1341.7396,   82.5520, 1365.9414,  101.7065]], device='cuda:0')), scores: ten